In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load the data
housing_data = pd.read_csv('/content/Melbourne_housing_FULL.csv')

# Calculate missing percentage and remove columns with more than 20% missing values, except 'Price'
missing_percentage = housing_data.isnull().mean() * 100
columns_to_remove = [column for column in missing_percentage.index if missing_percentage[column] > 20 and column != 'Price']
cleaned_data = housing_data.drop(columns=columns_to_remove)

# Drop rows with missing 'Price'
cleaned_data = cleaned_data.dropna(subset=['Price'])

# Define features and target
# Exclude 'Address' from features as it's not numerical and causes the error
features = cleaned_data.drop(columns=['Price', 'Date', 'Suburb', 'Type', 'Method', 'SellerG', 'CouncilArea', 'Regionname', 'Address'])
target = cleaned_data['Price']

# Fill missing values in numeric columns with mean
numeric_features = features.select_dtypes(include=[float, int])
features[numeric_features.columns] = numeric_features.fillna(numeric_features.mean())

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Remove highly correlated features (correlation > 0.85)
correlation_matrix = X_train.corr().abs()
high_correlation_pairs = np.where(correlation_matrix > 0.85)
high_correlation_features = set([X_train.columns[i] for i in high_correlation_pairs[0] if i != high_correlation_pairs[1][i]])  # Remove diagonal

# Drop highly correlated features from training and test data
X_train_filtered = X_train.drop(columns=high_correlation_features)
X_test_filtered = X_test.drop(columns=high_correlation_features)

# Define a function to evaluate model performance
def evaluate_model(train_features, test_features, train_target, test_target):
    model = RandomForestRegressor(random_state=42)
    model.fit(train_features, train_target)
    predictions = model.predict(test_features)
    mse = mean_squared_error(test_target, predictions)
    return mse

# Evaluate the model after removing highly correlated features
mse_after_correlation_filter = evaluate_model(X_train_filtered, X_test_filtered, y_train, y_test)
print(f"Model performance after removing highly correlated features: MSE = {mse_after_correlation_filter:.2f}")

Model performance after removing highly correlated features: MSE = 143875373039.63
